# Agent Short-Term Memory

Short-term memory stores conversation history within a session using checkpointers.

**What you'll learn:**
- Checkpointers persist conversation history
- SQLite for development, PostgreSQL for production
- Thread IDs manage separate sessions
- Access agent state in tools with ToolRuntime
- Modify agent state from tools for context offloading
- Save/load conversation summaries for long conversations

## Checkpointer Comparison

| Type | Use Case | Setup |
|------|----------|-------|
| **SQLite** | Development, testing | Simple file-based |
| **PostgreSQL** | Production, multi-user | Database connection |

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, ToolMessage, SystemMessage
from langchain.tools import tool, ToolRuntime
from pydantic import BaseModel
from pathlib import Path
from langgraph.types import Command
from scripts import base_tools

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

system_prompt = """You are a helpful assistant with memory.
- Remember previous messages in the conversation
- Use conversation history when answering questions
- Be concise and accurate"""

## Problem: No Memory

In [5]:
agent = create_agent(model=model)

agent.invoke({'messages': [HumanMessage("My name is John")]})
response = agent.invoke({'messages': [HumanMessage("What's my name?")]})

response

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='d9806100-b985-43f4-94a2-322f9fcf67a7'),
  AIMessage(content="I don't know your name. As an AI, I don't have access to personal information about you or a memory of who you are from past interactions.\n\nIf you'd like me to know your name for our current conversation, you're welcome to tell me!", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba188-3665-7f31-8b22-4bcb08d73614-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 514, 'total_tokens': 521, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 456}})]}

## Short-Term Memory: SQLite

In [6]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

os.makedirs("db", exist_ok=True)

conn = sqlite3.connect("db/checkpoints.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)
checkpointer.setup()

agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    checkpointer=checkpointer,
    system_prompt=system_prompt
)

In [7]:
config = {"configurable": {"thread_id": "user_123"}}

agent.invoke({"messages": [HumanMessage("My name is John")]}, config)
response = agent.invoke({"messages": [HumanMessage("What's my name?")]}, config)

response

{'messages': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}, id='1b83b791-9bd6-4d99-a6f5-5422afea342a'),
  AIMessage(content='Hello John! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba188-6ee9-7f52-8fce-a760b242659d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 236, 'output_tokens': 10, 'total_tokens': 246, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='a423a099-597c-4d00-8104-2235aa93d7df'),
  AIMessage(content=[{'type': 'text', 'text': 'Your name is John.', 'extras': {'signature': 'Cr8BAXLI2nxqrhibbTHVglMpNh2Ix8fXHOS+sxUaPnmyI1pzXWhYiUS0+gmvyztpdqslCfHtXZ98stvY1ENrCLI8x5b7f6kHXoH+q4ncb6GkB2MmgfdDJGlkbj5WQDxD4i7LPD1vEBGoAARlGHmMfUFBdMEcENisxkFTV15GuV7YNSdhciRX767L4TVt9BOrtPalBGQ7o5T

In [7]:
response

{'messages': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}, id='ec1e622c-a5af-4cb3-ba57-eae9e0722177'),
  AIMessage(content='Hello John! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba185-d33a-7ab2-ae09-af12af9d9bf9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 206, 'output_tokens': 10, 'total_tokens': 216, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='b94b3ba3-76df-4b54-9993-2833f920e608'),
  AIMessage(content="I'm sorry, I don't remember your name. Could you please remind me?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba185-d71c-7101-939a-a9dcabaf8dea-

## Short-Term Memory: PostgreSQL

In [ ]:
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg

with PostgresSaver.from_conn_string(os.getenv("POSTGRESQL_URL")) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[base_tools.web_search, base_tools.get_weather],
        checkpointer=checkpointer
    )
    
    config = {"configurable": {"thread_id": "postgres_session"}}
    
    agent.invoke({'messages': [HumanMessage("My city is Mumbai")]}, config)
    response = agent.invoke({'messages': [HumanMessage("What's my city?")]}, config)
    
    response['messages'][-1].content

## Context Offloading: Read State in Tools

Access agent state to save conversation summaries for context management.

In [8]:
class CustomContext(BaseModel):
    user_id: str
    thread_id: str

@tool
def save_conversation_summary(summary: str, runtime: ToolRuntime[CustomContext]) -> str:
    """Save conversation summary to disk for context offloading."""
    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id
    
    # Create directory structure
    summary_dir = Path(f"data/{user_id}/{thread_id}")
    summary_dir.mkdir(parents=True, exist_ok=True)
    
    # Write summary
    summary_path = summary_dir / "summary.md"
    summary_path.write_text(summary)
    
    return f"Summary saved to {summary_path}"

In [9]:
# Test save summary
agent = create_agent(
    model=model,
    tools=[save_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

config = {"configurable": {"thread_id": "session1"}}

response = agent.invoke({
    "messages": [HumanMessage("Save this summary: User discussed Python and AI topics")]
}, config=config, context=CustomContext(user_id="user_123", thread_id="session1"))

response['messages'][-1].content

'I have saved the summary of our conversation.'

## Context Offloading: Modify State in Tools

Load previous summaries and inject them into agent state.

In [10]:
@tool
def load_conversation_summary(runtime: ToolRuntime[CustomContext]) -> Command:
    """Load previous conversation summary from disk."""
    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id
    
    summary_path = Path(f"data/{user_id}/{thread_id}/summary.md")
    
    if not summary_path.exists():
        return Command(update={
            "messages": [
                ToolMessage(
                    "No previous summary found.",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        })
    
    # Read summary
    summary_text = summary_path.read_text()
    
    # Update state with summary as system message
    return Command(update={
        "messages": [
            SystemMessage(f"Previous conversation summary:\n{summary_text}"),
            ToolMessage(
                "Successfully loaded previous summary.",
                tool_call_id=runtime.tool_call_id
            )
        ]
    })

In [11]:
# Test load summary
agent = create_agent(
    model=model,
    tools=[load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

config = {"configurable": {"thread_id": "session1"}}

response = agent.invoke({
    "messages": [HumanMessage("Load my previous conversation summary")]
}, config=config, context=CustomContext(user_id="user_123", thread_id="session1"))

In [12]:
response

{'messages': [HumanMessage(content='Save this summary: User discussed Python and AI topics', additional_kwargs={}, response_metadata={}, id='1ee26e6d-b8b0-4ce3-bc94-f65e61ba9472'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'save_conversation_summary', 'arguments': '{"summary": "User discussed Python and AI topics"}'}, '__gemini_function_call_thought_signatures__': {'b4d1620d-9795-4ebd-bff1-496763b651e4': 'CpQCAXLI2nyHVcIWe/V9eAxm5z/TcRlBY+tyRIZBCMZlesn1NYXHj1R6a9TXUzHPZ9Gzxp1SqfaYJtLiPeQN8Yqj2fvnScfUK9F6pzNlwd1KmJYQdpEa5slHO/cKznwPhrTVGM00REyaGrzTz/3qR2ufCWkoNXByQoAK5InSMnfnXTAGJOBzfdxY/v1DK9otZmJburOsFEHuE0ZNqPPaWiPhiCsrm9f1FGtUHrB5rHb7FKfHZ5HEG+euGfsOISR6I7dkMohWqFVD21TgWhZYyZFsNMOSvuPbHx/+kBv5PsJZCaaBNpThEwo1BJ/G9Ew/dwzzEBw0PMXCfK7GrHx8h8xGUAbU2oNdGkH62o2mdPk2dGO87X+F'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba188-da1f-7812-9042-85ad0ffa3156-0',

## Combined: Save and Load Context

Real-world example combining both tools for context management.

In [13]:
# Agent with both save and load
agent = create_agent(
    model=model,
    tools=[save_conversation_summary, load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

# Session 1: Have conversation and save summary
config = {"configurable": {"thread_id": "project_alpha"}}

agent.invoke({
    "messages": [HumanMessage("We're building a chatbot using LangChain")]
}, config=config, context=CustomContext(user_id="user_456", thread_id="project_alpha"))

agent.invoke({
    "messages": [HumanMessage("Save summary: Building LangChain chatbot with memory and tools")]
}, config=config, context=CustomContext(user_id="user_456", thread_id="project_alpha"))

print("Summary saved for project_alpha")

Summary saved for project_alpha
